<a href="https://colab.research.google.com/github/aalonsca73/in_silico_toxicology/blob/main/in_silico_toxicology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install fastapi kaleido python-multipart uvicorn pubchempy rdkit mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [9]:
# Define the filename for the Excel file
filename = 'llistes.xlsx'

# Define the column name to be used as the index.
# Set this variable to the name of the column containing the molecule names of interest.
index = 'Name'

# Import necessary libraries
import sys
import pubchempy as pcp
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from mordred import Calculator, descriptors

# Create a descriptor calculator with all available descriptors
calc = Calculator(descriptors)

# Read the Excel file using pandas
df = pd.read_excel(filename,sheet_name=1,header=0)
df.set_index(index,inplace=True)

# Create a list to store all canonical SMILES
SMILES_list = []

# Create a list to store molecules not found in PubChem
molecules_not_found = []

# Iterate through each molecule identifier in the index column to retrieve compound names from PubChem
for ids in df.index:
    try:
        # Fetch the compound name from PubChem
        compound_name = pcp.get_compounds(ids,'name')
        # Check if compound name is empty (no results)
        if not compound_name:
            # Use boolean indexing to drop rows with empty compound_name
            df.drop(ids,axis=0,inplace=True)
            # Add the molecule to the list of molecules_not_found
            molecules_not_found.append(ids)
        else:
            # Extract the first identifier for the compound
            first_identifier = compound_name[0].cid
            # Fetch the canonical SMILES for the first identifier
            first_smiles = pcp.get_compounds(first_identifier,'cid')[0].canonical_smiles
            # Add the canonical SMILES to a list called SMILES_list
            SMILES_list.append(first_smiles)
    except Exception as e:
        print(f"Error for molecule {ids}: {e}")

# Create a DataFrame with molecules not found in PubChem
missing_molecules_df = pd.DataFrame(data=molecules_not_found)

# Reset the index after dropping rows
df.reset_index(inplace=True)

# Remove duplicate canonical SMILES while preserving order
canonical_SMILES = list(dict.fromkeys(SMILES_list))

# We create a DataFrame with canonical SMILES
smiles_df = pd.DataFrame(data=canonical_SMILES)
# Rename the column
smiles_df.columns = ['SMILES']
# Insert the DataFrame with compound names into the DataFrame with SMILES
smiles_df.insert(0,'Name',df['Name'],True)

# Create a new list called data to store all properties
data = []

# Iterate through every SMILES in smiles_df to extract properties of every molecule
for molecule in smiles_df['SMILES']:
    try:
        mol = Chem.MolFromSmiles(molecule)
        data.append(mol)
    except:
        print(molecule)

# Create a new DataFrame called props_df with all obtained molecular properties
props_df = calc.pandas(data)

# Remove molecular descriptors that are not informative
# Delete columns where all values are zero
props_df = props_df.loc[:, ~np.all(np.equal(props_df.values, 0), axis=0)]
# Delete columns containing non-numeric data
props_df = props_df.select_dtypes(include=[np.number])

# Merge the DataFrame containing SMILES with the DataFrame containing all properties
names_props = pd.concat([smiles_df,props_df],axis=1)

# Create a new document with all information
names_props.to_csv('molecules_with_properties.csv',index=False)

# Create another document with molecules not found in PubChem
missing_molecules_df.to_csv('molecules_not_found.csv',index=False)

100%|██████████| 65/65 [00:16<00:00,  3.93it/s]
